### 앙상블
#### 결정트리를 기반으로 만들어진 모델들
#### 기존 하나의 결정트리를 사용했더면 앙상블 모델은 여러개의 결정트리를 사용한다고 이해

#### [앙상블 모델 - 분류와 회귀 모두 가능, 모두 랜덤포레스트를 기반으로 조금씩 성능을 개선한 모델임]
#### - 랜덤포레스트(RandomForest)
#### - 엑스트라트리(Extra Trees) 
#### - 그라디언트 부스팅(Gradient Boosting)
#### - 히스토그램 기반 그라디언트 부스팅(Histogram-based Graidient Boosting)

#### - (분류로만 가능한 모델)
#### - LightGBM, XGBoosting
---

### RandomForest
#### 앙상블 학습 모델 중 가장 대표적인 모델로 가장 먼저 시작하는 학습모델임
#### 훈련데이터에서 과대적합되는 것을 막아줌
####  검증데이터와 테스트데이터에서 안정적인 성능을 얻을수 있음

#### [ 학습개념 ]
#### - 결정트리 여러개를 이용함
#### -여러개의 결정트리를 랜덤하게 만듦
#### -훈련 시 랜덤하게 훈련데이터 샘플을 추출
#### -사용된 훈련데이터 샘플을 반환하여 다시 랜덤하게 훈련데이터를 추출하여 훈련

#### [ 훈련데이터 사용순서 ]
#### - 데이터 원본에서 중복을 허용하여 데이터를 샘플링함
#### - 원본에서 랜덤하게 샘플 추출
#### - 사용이 끝난 샘플을 원본에 반환
#### -다시 랜덤하게 샘플을 추출
#### -원본에서 랜덤하게 샘플을 반환하면서 추출하기에 훈련에 사용되었던 값들이 다시 추출될수 있음 -> 이러한 훈련데이터 흐름을 "부트스트랩 샘플링" 이라고 칭함

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

### 데이터 수집/전처리/분리

In [3]:
df = pd.read_csv('./data/08_wine.csv')
df.head(3)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0


In [4]:
x = df.drop('class', axis=1).to_numpy()
y = df['class'].to_numpy()

In [11]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x,y, random_state=42, test_size=0.2)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((5197, 3), (1300, 3), (5197,), (1300,))

### model

In [7]:
rf = RandomForestClassifier()
rf.fit(train_x, train_y)


RandomForestClassifier()

In [19]:
print('rf train :',rf.score(train_x, train_y)*100)
print('rf test :',rf.score(test_x, test_y)*100)

rf train : 99.6921300750433
rf test : 88.76923076923077


In [104]:
## cross validate( default fold =5)
# train_score : 훈련정확도
# test_score : 검증정확도
# return_train_score : 훈련정확도를 포함해서 조회하고자 할 때 True

cv_rf = cross_validate(rf, train_x, train_y, return_train_score=True)
cv_rf

{'fit_time': array([0.20570993, 0.21872401, 0.22430611, 0.22487068, 0.21474528]),
 'score_time': array([0.01561999, 0.01562452, 0.01295686, 0.01395321, 0.01937938]),
 'test_score': array([0.88173077, 0.88173077, 0.91241578, 0.89797883, 0.88450433]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}

In [105]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_rf train :',np.mean(cv_rf['train_score'])*100)
print('cv_rf test :',np.mean(cv_rf['test_score'])*100)

cv_rf train : 99.74503966084433
cv_rf test : 89.16720959502481


### RandomForest 독립변수 중요도 - feature_importances_

In [38]:
print(df.drop('class', axis=1).columns)
print(rf.feature_importances_)

Index(['alcohol', 'sugar', 'pH'], dtype='object')
[0.23206665 0.49968101 0.26825234]


#### RandomForest 특징
#### - 훈련데이터를 랜덤하게 사용하다 보면 훈련에 참여하지 못한느 값들이 있음
#### - 훈련에 참여하지 못한 값들을 이용해서 다시 교차검증을 할 수 있음 - oob(Out of Bag)옵션을 적용하여 훈련 수행

#### - oob_score : 검증데이터용으로 사용함

In [39]:
orf = RandomForestClassifier(oob_score=True, random_state=42)
orf.fit(train_x, train_y)

RandomForestClassifier(oob_score=True, random_state=42)

In [40]:
print('orf train :',orf.score(train_x, train_y)*100)
print('orf test :',orf.score(test_x, test_y)*100)

orf train : 99.6921300750433
orf test : 88.92307692307693


### ExtraTrees 모델
#### 램덤포레스트와 유사하게 작동되며 기본적으로 100개의 결정트리로 훈련시작

#### [ 랜덤포레스트와 차이점 ]
#### - 랜덤포레스트는 부트스트랩 샘플링기법으로 중복을 허용하나
#### - 엑스트라트리는 훈련데이터 전체를 이용해 결정트리를 생성함
#### - 무작위로 트리로 분할함

#### [ 특징 ]
#### - 과대적합을 막고 검증데이터의 정확도를 높일 수 있음
#### - 하지만, 특성이 많지 않은 경우 램덤포레스트와 큰 차이가 없음
#### - 랜덤포레스트보다 무작위성이 크기 때문에 더 많은 결정트리를 훈련시킴
#### - 랜덤하게 트리를 분할하기 때문에 계산 속도가 빠른 장점이 있음

In [41]:
et = ExtraTreesClassifier()
et.fit(train_x, train_y)

ExtraTreesClassifier()

In [48]:
print('et train :',et.score(train_x, train_y)*100)
print('et test :',et.score(test_x, test_y)*100)

et train : 99.6921300750433
et test : 87.76923076923076


In [49]:
# 훈련중요도 확인
et.feature_importances_

array([0.20168721, 0.51654299, 0.2817698 ])

In [101]:
cv_et = cross_validate(et, train_x, train_y, return_train_score=True)
cv_et

{'fit_time': array([0.1731286 , 0.17186165, 0.17201114, 0.18995667, 0.17980647]),
 'score_time': array([0.01562309, 0.01562023, 0.01566553, 0.01694322, 0.01562691]),
 'test_score': array([0.88365385, 0.87980769, 0.89894129, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}

In [102]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_et train :',np.mean(cv_et['train_score'])*100)
print('cv_et test :',np.mean(cv_et['test_score'])*100)

cv_et train : 99.74503966084433
cv_et test : 88.76297475383134


### Gradient Boosting
#### 트리의 깊이가 얕은 결정트리를 사용 하며 기본적으로 max_depth가 3임
#### 트리의 갯수는 기본적으로 100
#### 기존에 다른 훈련모델의 결과가 좋지 않을 경우에 사용해보면 됨
#### - 또는 훈련모델들의 결과가 좋지 않을 경우에 비교대상으로 사용
#### 과적합해소에 강하며, 일반화(과대/과소 최적화)에 강함

#### [성능향상 테스트 방법]
#### - 결정트리의 갯수를 늘려가면서 훈련테스트
#### - 학습률을 지원하기 때문에 학습률의 값을 증가시켜본다(기본값 =  0.1)

#### [단점]
#### - 순서대로 트리를 추가합(랜덤하지 않음)
#### - 랜덤하지 않기 때문에 속도가 다소 느린 단점이 있음
#### - 느린 단점을 보완한 모델이 "히스토그램 기반 그레디언트 부스팅"이 있음

In [53]:
gb = GradientBoostingClassifier(random_state=42)
gb.fit(train_x, train_y)

GradientBoostingClassifier(random_state=42)

In [55]:
print('gb train :',gb.score(train_x, train_y)*100)
print('gb test :',gb.score(test_x, test_y)*100)

gb train : 88.80123147969982
gb test : 86.6923076923077


In [57]:
"""교차검증 """
cv_gb = cross_validate(gb, train_x, train_y, return_train_score=True)
cv_gb

{'fit_time': array([0.18769145, 0.18755078, 0.18752217, 0.18747663, 0.1874876 ]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.86634615, 0.87019231, 0.89412897, 0.86044273, 0.86910491]),
 'train_score': array([0.89006495, 0.88958383, 0.88239538, 0.89249639, 0.88600289])}

In [60]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_gb rf train :',np.mean(cv_gb['train_score'])*100)
print('cv_gb rf test :',np.mean(cv_gb['test_score'])*100)

cv_gb rf train : 88.81086892152564
cv_gb rf test : 87.20430147331015


### [해석] 과적합이 다른 모델보다 좋아짐

### 성능 튜닝 - 결정트리 갯수 조정

In [59]:
""" 결정트리를 500개로, 학습률은 0.2로"""
tun_gb = GradientBoostingClassifier(n_estimators=500, 
                                   learning_rate=0.2,
                                   random_state=42)
tun_gb.fit(train_x, train_y)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=500, random_state=42)

In [65]:
print('tun_gb train :',tun_gb.score(train_x, train_y)*100)
print('tun_gb test :',tun_gb.score(test_x, test_y)*100)

tun_gb train : 93.8233596305561
tun_gb test : 87.07692307692308


In [66]:
"""특성중요도"""
tun_gb.feature_importances_

array([0.15853457, 0.68010884, 0.1613566 ])

In [63]:
""" 교차검증 """
cv_tun_gb = cross_validate(tun_gb, train_x, train_y, return_train_score=True)
cv_tun_gb

{'fit_time': array([0.91182399, 0.93447781, 0.89576578, 0.92025757, 0.92233324]),
 'score_time': array([0.01562309, 0.        , 0.01562333, 0.0049839 , 0.00498343]),
 'test_score': array([0.875     , 0.87211538, 0.89701636, 0.8719923 , 0.87391723]),
 'train_score': array([0.9494828 , 0.94443108, 0.94468494, 0.94324194, 0.95045695])}

In [64]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_tun_gb train :',np.mean(cv_tun_gb['train_score'])*100)
print('cv_tun_gb test :',np.mean(cv_tun_gb['test_score'])*100)

cv_tun_gb train : 94.64595437171815
cv_tun_gb test : 87.80082549788999


### Histogram-Based Gradient Boosting
#### 그라디언트 부스팅 모델의 느린 속도를 개선한 모델
####  정형 데이터를 다루는 머신러닝 모델 중 인기가 높음

#### [속도가 빠른 이유]
#### - 최초에 특성을 256개의 구간으로 미리 나누어 놓고 시작
#### - 트리를 분할 할때 매우 빠르게 찾을수 있음
#### [ 성능향상 방법]
#### - max-iter 매개변수의 값을 변경하여 테스트 진행
#### - 기본적으로 기본값을 이용해도 안정적인 성능을 얻을 수 있음

In [83]:
ht = HistGradientBoostingClassifier(random_state=42)
ht.fit(train_x, train_y)

HistGradientBoostingClassifier(random_state=42)

In [87]:
print('ht train :',ht.score(train_x, train_y)*100)
print('ht test :',ht.score(test_x, test_y)*100)

ht train : 92.4956705791803
ht test : 87.23076923076924


In [89]:
"""히스토그램 그라디언트는 특성 중요도가 없음"""

'히스토그램 그라디언트는 특성 중요도가 없음'

In [90]:
# 예측하기
ht.predict(test_x)

array([1., 0., 1., ..., 1., 1., 1.])

In [ ]:
""" 정확도 값의 비교 
훈련정확도 > 검증정확도 > 테스트정확도"""

In [85]:
""" 교차검증 """
cv_ht = cross_validate(ht, train_x, train_y, return_train_score=True)
cv_ht

{'fit_time': array([0.23095965, 0.17185664, 0.18162084, 0.17185712, 0.1802578 ]),
 'score_time': array([0.        , 0.01562333, 0.        , 0.        , 0.        ]),
 'test_score': array([0.87115385, 0.88365385, 0.90279115, 0.86621752, 0.87680462]),
 'train_score': array([0.93192206, 0.93216262, 0.92857143, 0.93265993, 0.93554594])}

In [92]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_ht train :',np.mean(cv_ht['train_score'])*100)
print('cv_ht test :',np.mean(cv_ht['test_score'])*100)

cv_ht train : 93.21723946453318
cv_ht test : 88.01241948619236


### XGBoosting
#### - 사이킷런이 아닌 별도 라이브러리로 제공
#### - pip install xgboost로 설치해야 함
#### - tree_method : 사용할 모델을 지정해야함

In [91]:
# 라이브러리
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [99]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
xgb.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [100]:
print('xgb train :',xgb.score(train_x, train_y)*100)
print('xgb test :',xgb.score(test_x, test_y)*100)

xgb train : 95.0740812006927
xgb test : 86.84615384615385


In [109]:
""" 교차검증 """
cv_xgb = cross_validate(xgb, train_x, train_y, return_train_score=True, n_jobs=-1)
cv_xgb

{'fit_time': array([0.07811618, 0.07811618, 0.07811618, 0.06249285, 0.07811618]),
 'score_time': array([0.        , 0.        , 0.        , 0.01562333, 0.        ]),
 'test_score': array([0.8625    , 0.8875    , 0.89797883, 0.87969201, 0.8719923 ]),
 'train_score': array([0.95814289, 0.9569401 , 0.95526696, 0.95286195, 0.95430495])}

In [97]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_xgb train :',np.mean(cv_xgb['train_score'])*100)
print('cv_xgb test :',np.mean(cv_xgb['test_score'])*100)

cv_xgb train : 95.55033709953123
cv_xgb test : 87.99326275264677


### LightGBM
#### 마이크로소프트에서 만든 히스토그램 기반 그라디언트 부스트 라이브러리
#### 훈련 속도가 빠름 - 사이킷런이 아닌 별도 라이브러리로 제공
#### 최신 기술을 많이 적용하고 있어서 인기가 높음
#### 사이킷런의 히스토그램 기반 그라디언트 부스팅 라이브러리가  LightGBM의 영향을 받아서 만들어졌다는 설이 있음

In [116]:
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(train_x, train_y)

LGBMClassifier(random_state=42)

In [117]:
print('lgbm train :',lgbm.score(train_x, train_y)*100)
print('lgbm test :',lgbm.score(test_x, test_y)*100)

lgbm train : 92.88050798537618
lgbm test : 87.3076923076923


In [118]:
""" 교차검증 """
cv_lgbm = cross_validate(lgbm, train_x, train_y, return_train_score=True, n_jobs=-1)
cv_lgbm

{'fit_time': array([0.04686952, 0.04686952, 0.03124619, 0.03124619, 0.04686952]),
 'score_time': array([0.01562548, 0.        , 0.01562333, 0.01562333, 0.        ]),
 'test_score': array([0.87307692, 0.87692308, 0.91049086, 0.86044273, 0.87969201]),
 'train_score': array([0.93841713, 0.93432764, 0.93410293, 0.93434343, 0.93795094])}

In [119]:
# 폴드별 훈련성능을 평균한 훈련 정확도
print('cv_lgbm train :',np.mean(cv_lgbm['train_score'])*100)
print('cv_lgbm test :',np.mean(cv_lgbm['test_score'])*100)

cv_lgbm train : 93.5828414851749
cv_lgbm test : 88.01251203079885


### 6개 모델 교차검증 성능비교

In [115]:
print('------'*3,'RandomForest','------'*3)
print('cv_rf train :',np.mean(cv_rf['train_score'])*100)
print('cv_rf test :',np.mean(cv_rf['test_score'])*100)
print()
print('------'*3,'ExtraTrees','------'*3)
print('cv_et train :',np.mean(cv_et['train_score'])*100)
print('cv_et test :',np.mean(cv_et['test_score'])*100)
print()
print('------'*3,'Gradient Boosting','------'*3)
print('cv_tun_gb train :',np.mean(cv_tun_gb['train_score'])*100)
print('cv_tun_gb test :',np.mean(cv_tun_gb['test_score'])*100)
print()
print('------'*3,'Histigram-Based Gradient Boosting','------'*3)
print('cv_ht train :',np.mean(cv_ht['train_score'])*100)
print('cv_ht test :',np.mean(cv_ht['test_score'])*100)
print()
print('------'*3,'XGBoosting','------'*3)
print('cv_xgb train :',np.mean(cv_xgb['train_score'])*100)
print('cv_xgb test :',np.mean(cv_xgb['test_score'])*100)
print()
print('------'*3,'LightGBM','------'*3)
print('cv_lgbm train :',np.mean(cv_lgbm['train_score'])*100)
print('cv_lgbm test :',np.mean(cv_lgbm['test_score'])*100)

------------------ RandomForest ------------------
cv_rf train : 99.74503966084433
cv_rf test : 89.16720959502481

------------------ ExtraTrees ------------------
cv_et train : 99.74503966084433
cv_et test : 88.76297475383134

------------------ Gradient Boosting ------------------
cv_tun_gb train : 94.64595437171815
cv_tun_gb test : 87.80082549788999

------------------ Histigram-Based Gradient Boosting ------------------
cv_ht train : 93.21723946453318
cv_ht test : 88.01241948619236

------------------ XGBoosting ------------------
cv_xgb train : 95.55033709953123
cv_xgb test : 87.99326275264677

------------------ LightGBM ------------------
cv_lgbm train : 93.5828414851749
cv_lgbm test : 88.01251203079885


In [23]:
lll = np.arange(1,10).tolist()

In [77]:
type(lll)
lll

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [81]:
llll = [np.arange(1,10)]
#llll = np.arange(1,10)

In [82]:
type(llll)
llll

[array([1, 2, 3, 4, 5, 6, 7, 8, 9])]